will learn:
1. Agent Workflows
2. Use of tools to call functions
3. Agents collaboration via Tools and Handoffs

In [1]:
import os
import requests
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")
else:
    print(f"OpenAI API exists: {openai_api_key[:10]}...")

OpenAI API exists: sk-proj-Yq...


In [2]:
TESTMAIL_API_KEY = os.getenv("TESTMAIL_API_KEY")
NAMESPACE = "s65af"
BASE_URL = "https://api.testmail.app/api/json"

if not TESTMAIL_API_KEY:
    raise ValueError("TESTMAIL_API_KEY is not set in the environment variables.")
else:
    print(f"TestMail API exists: {TESTMAIL_API_KEY[:10]}...")

TestMail API exists: 94e4599d-a...


### Step 1: Agent workflow

In [3]:
instructions1 = """You are a sales agent working for CompIAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails."""

instructions_2 = """
You are a humorous, engaging sales agent working for CompIAI.
CompIAI provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.
Write witty, engaging cold emails that are likely to get a response.
"""

instructions_3 = """
You are a busy sales agent working for CompIAI.
CompIAI provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.
Write concise, to the point cold emails.
"""

In [4]:
sales_agent_1 = Agent(
    name="CompIAI Sales Agent 1",
    instructions=instructions1,
    model="gpt-5-nano",
)

sales_agent_2 = Agent(
    name="CompIAI Sales Agent 2",
    instructions=instructions_2,
    model="gpt-5-nano",
)

sales_agent_3 = Agent(
    name="CompIAI Sales Agent 3",
    instructions=instructions_3,
    model="gpt-5-nano",
)

In [5]:
result = Runner.run_streamed(sales_agent_1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Accelerate SOC 2 readiness with AI-powered automation

Hi [First Name],

SOC 2 readiness can be a slog—evidence gathering, control mapping, and drafting audit artifacts can derail your roadmap. CompIAI is an AI-powered SaaS platform designed to simplify audit prep and ongoing compliance.

With CompIAI, you get:
- AI-assisted evidence collection and control mapping to the SOC 2 Trust Services Criteria
- A centralized evidence repository and audit-ready reports
- A continuous readiness score that highlights gaps and priorities
- Automated evidence generation for auditors and reviewer access

This helps teams shorten prep times, reduce last-minute gaps, and stay audit-ready year-round.

Would you have 15 minutes this week for a quick, tailored demo to show how it would fit your stack and timeline? If yes, what time works for you?

Best regards,
[Your Name]
[Title], CompIAI
[Phone] | [Email] | [Website]

P.S. I can share a 2-page readiness plan based on your current SOC 2 scope.

## Concurrent execution of multiple agents

In [6]:
message = "Write a cold sales email"
with trace("Concurrent Sales Agents: cold email"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1, message),
        Runner.run(sales_agent_2, message),
        Runner.run(sales_agent_3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Accelerate SOC 2 readiness at {Company} with AI-powered automation

Hello {FirstName},

I noticed {Company} is preparing for SOC 2, and I know how time-consuming evidence gathering, control mapping, and audit readiness can be across multiple tools and teams.

CompIAI is an AI-powered SaaS platform designed to streamline SOC 2 compliance and audit prep. Our system automates evidence collection, maps controls to the SOC 2 criteria, and provides ready-to-submit audit packs along with guided remediation.

What you get with CompIAI:
- Continuous monitoring and auto evidence collection from cloud services, IAM, CI/CD, and ticketing systems
- Flexible control mapping to SOC 2 criteria with auditable trails
- Ready-to-submit audit packs, policy templates, and versioned documentation
- Gap analysis, risk scoring, and remediation workflows to close findings faster
- An organized evidence library with exportable reports for auditors

If you’re open to a quick 15-minute chat, I can tailor

In [7]:
sales_picker_agent = Agent(
    name="Sales Email Picker Agent",
    instructions="""
    Pick the best cold sales email from the given options.
    Imagine you are a customer and pick the one you are most likely to respond to.
    Do not give an explanation; reply with the selected email only.
    """,
)

In [8]:
message = "write a cold sales email"
with trace("Sales Email Picker Agent: pick best email"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1, message),
        Runner.run(sales_agent_2, message),
        Runner.run(sales_agent_3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n" + "\n\n".join(outputs)

    best_email = await Runner.run(sales_picker_agent, emails)

    print(f"Best email:\n{best_email.final_output}")

Best email:
Subject: SOC 2 ready in days, not months — meet CompIAI

Hi [FirstName],

SOC 2 prep feeling like a never-ending scavenger hunt? You’re not imagining it. We built CompIAI to automate the boring, repetitive parts of readiness so your team can actually focus on securing your systems—and shipping features.

What CompIAI does for you:
- AI-driven control mapping that aligns to your environment and policies
- Continuous evidence collection and live readiness dashboards
- Audit-ready artifacts and reports you can share in minutes (not days)
- Clear remediation tracking so you dodge last-minute audit surprises

All of this sits behind an easy-to-use UI that your SOC team will actually enjoy using.

Would you be open to a quick 15-minute chat this week? I’ll tailor a short demo to your controls and show a real-world example from a company like yours.

Cheers,
[Your Name]
[Title], CompIAI
[Phone] | [Website]
P.S. If you’d rather not hop on a call, I can send a 60-second demo video o